In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import random
from pytorch_lightning.loggers import TensorBoardLogger
import json
import sys
sys.path.append('..')

import os
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as T
import warnings
import pickle
warnings.filterwarnings("ignore")

from src.data_assemble.wrap_data import *
from src.models.WindCNN import *
from src.data_assemble.wrap_data import *

# %load_ext tensorboard
# %tensorboard --logdir lightning_logs/
random.seed(24)

In [12]:

path_to_data = os.path.join('..', 'data', 'nn_data')
st_split_dict = train_val_test_split(path_to_data, train = 0.5, val = 0.25, test = 0.25, verbose = True)

{'Train': ['Туапсе', 'Краснодар, Круглик'], 'Val': ['Анапа'], 'Test': ['Приморско-Ахтарск']}


In [13]:
path_to_dump = os.path.join('..', 'data','nn_data')
X, y = extract_splitted_data(path_to_dump, st_split_dict)

In [14]:
batch_size = 1024
with open(os.path.join('..', 'conf', 'conv_config.json')) as fs:
    args = json.load(fs)

In [16]:
logger = TensorBoardLogger(save_dir='../logs/wind', name='windnet')
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_auroc", min_delta=0.00, patience=5, verbose=False, mode="max")
trainer = pl.Trainer(max_epochs=500,
                    gpus='1',
                    benchmark=True,
                    check_val_every_n_epoch=1,
                    callbacks=[early_stop_callback]
)


dm = WindDataModule(X=X, y=y, batch_size=batch_size, downsample=False)
model = WindNetPL(args)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type            | Params
-------------------------------------------------
0 | net          | WindNet         | 23.0 K
1 | accuracy     | Accuracy        | 0     
2 | AUROC        | AUROC           | 0     
3 | precision_m  | Precision       | 0     
4 | recall       | Recall          | 0     
5 | F1           | F1Score         | 0     
6 | conf_matrix  | ConfusionMatrix | 0     
7 | stats_scores | StatScores      | 0     
8 | loss_f       | NLLLoss         | 0     
-------------------------------------------------
23.0 K    Trainable params
0         Non-trainable params
23.0 K    Total params
0.092     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [7]:
trainer.test(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_AUROC_step        0.6094663143157959
      test_acc_step         0.9937020540237427
      test_f1_step          0.9968386888504028
        test_loss           0.6616266244811383
   test_precision_step      0.9937020540237427
    test_recall_step                1.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6616266244811383,
  'test_acc_step': 0.9937020540237427,
  'test_recall_step': 1.0,
  'test_AUROC_step': 0.6094663143157959,
  'test_precision_step': 0.9937020540237427,
  'test_f1_step': 0.9968386888504028}]